# Deconvolution
For already mean combined, telescope and unresolved corrected data

Correction for PSF smearing will be after! 

16.08.2024 - I do not use boundary condition any more and use the cutted version of the PSF as it allows to avoid niases caused by detector features in the PSF image that are not part of the actual star image. I also use a region of 112 pixels (~400 mas) from the center to calculate difference between iterations 


Current set up is for ZIMPOL data, but can be used for IRDIS as well by changing path, filenames and mas/pixel value


In [1]:
import matplotlib.pyplot as plt
import numpy as np
import fnmatch
import os
from astropy.io import fits

from scipy.signal import convolve2d
from scipy.signal import convolve, oaconvolve
from PIL import Image, ImageChops
from astropy.convolution import convolve_fft

import sys

## Functions

In [2]:
def normalize(arr):
    rng = arr.max()-arr.min()
    amin = arr.min()
    return (arr-amin)*1/rng #normalised to [0,1]

def compare(img1, img2):
    # normalize to compensate for exposure difference
    
    img1 = normalize(img1)
    img2 = normalize(img2)
    # calculate the difference and its norms
    diff = img1 - img2  # elementwise for scipy arrays
    m_norm = np.sum(abs(diff))/np.sum(img1)  

    return m_norm





def Loadimages(star,fittype,dirdat,band, extension):
    dir = dirdat
    qfile = star+'_'+band+'_'+fittype+extension+'.fits'
    print(dir,qfile)
    print(qfile)
    files = os.listdir(dir)
    image=[]
    for file in files:
        if fnmatch.fnmatch(file, qfile):
            
            with fits.open(dir + file) as hdul:
                image = hdul[0].data
                print(image[1,4])
                n = image.shape[0]
                
    return image, n

def Loadimagespsf(star,dirdat,band):
    dir = dirdat
    qfile = '*_'+band+'_I_meancombined.fits'
    files = os.listdir(dir)
    image=[]
    for file in files:
        if fnmatch.fnmatch(file, qfile):
            with fits.open(dir + file) as hdul:
                image = hdul[0].data
                n = image.shape[0]

                print(n)
    return image, n




def plotImage(image, lim):
    n = image.shape[0]
    
    fig, ax = plt.subplots()
    image = np.arcsinh(image)
    max = np.max(image[int(n/2-lim/2):int(n/2+lim/2),int(n/2-lim/2):int(n/2+lim/2)])
    min=np.min(image[int(n/2-lim/2):int(n/2+lim/2),int(n/2-lim/2):int(n/2+lim/2)])
    ps = 3.6 #mas per pixel for IRDIS
    d = n * ps / 2
    plt.imshow(image, vmin=min, vmax=max, extent=(-d, d, d, -d))
    #plt.plot(0, 0, "+", color="red")
    plt.xlim(-lim * ps, lim * ps)
    plt.ylim(-lim * ps, lim * ps)
    plt.xlabel('mas')
    plt.ylabel("mas")
    plt.colorbar()
    plt.tight_layout

## Star Set up

In [3]:
#dirdat = '/media/kateryna/Data_Lin/PhD/SPHERE_reduction_data/'
#fig_folder= '/media/kateryna/Data_Lin/PhD/SPHERE_reduction_data/'

stars=['AR_Pup_dc_notnorm','HR4049_20190108','HR4049_20190107',"HR4049_combined",'V709_Car','HR4226','UMon_Katya']
stars=["V709_Car"]

#stars = ['IRAS08544-4431','IRAS08544-4431_dc_notnorm','UMon']


stars_type={'HD83878':'ref','IRAS08544-4431_dc_notnorm':'sci','HD75885':'ref', 'AR_Pup_dc_notnorm':'sci','UMon':'sci','UMon_Katya':'sci','HR4049_20190108':'sci','HR4049_20190107':'sci','HD71253':'ref','HD94680':'ref','HD96314':'ref','HD98025':'ref','V709_Car':'sci','HR4226':'sci'}
#ref_of_sci={'AR_Pup_dc_notnorm':'HD75885','UMon':'HD71253','UMon_Katya':'HD71253','V709_Car':'HD94680','HR4049_20190108':'HD96314','HR4049_20190107':'HD96314','HR4226':'HD98025',"HR4049_combined":'HD96314'} #the corresponding scientific targets, SPHERE proposals
ref_of_sci={'AR_Pup_dc_notnorm':'HD75885','UMon':'HD71253','UMon_Katya':'HD71253','V709_Car':'HD94680','HR4049_20190108':'HD96314','HR4049_20190107':'HD96314','HR4226':'HD98025',"HR4049_combined":'HD96314'} #the corresponding scientific targets, SPHERE proposals



fittypes=['I','Q_phi','PI']
#otherfittype=['I_pol']

bands=['I','V']

N_decon = 200


## Deconvolution

In [4]:
dirdat0 = '/media/kateryna/Data_Lin/PhD/SPHERE_reduction_data/paper3/' 


for star in stars:
    print(star)
    critlim=0.015
    if star =='V709_Car': critlim=0.05 

    dirdat =dirdat0+'Unres+PSFcorr/'+star+'/'

    for band in bands:

        dirName1=dirdat0+'Deconvolution_corr_tel+unres'+'/'
        try:
        # Create target Directory
           os.mkdir(dirName1)
        except FileExistsError:
            print("Directory " , dirName1 ,  " already exists")

        dirName2=dirName1+star+'/'
        try:
        # Create target Directory
           os.mkdir(dirName2)
        except FileExistsError:
            print("Directory " , dirName2 ,  " already exists")


        #dirName2#=dirName1 #for creating folders for the second half of deconvolution (for the u_phi and i_pol)

        for fittype in fittypes:
            dirName=dirName2+'deconvolved_'+fittype+'/'
            try:
            # Create target Directory
               os.mkdir(dirName)
            except FileExistsError:
                print("Directory " , dirName ,  " already exists")
            image=[]
            PSF=[]
            ps=3.6
            
            if fittype=='I':
                image, n=Loadimages(star,fittype,dirdat,band,'_meancombined')
            elif fittype=='PSF':
                image,n=Loadimagespsf(ref_of_sci[star],'/media/kateryna/Data_Lin/PhD/SPHERE_reduction_data/paper3/mean_combined/'+ref_of_sci[star]+'/',band)          
            else:
                image, n=Loadimages(star,fittype,dirdat,band,'_corr_tel+unres') 
            
            #image, n=Loadimages(star,fittype,dirdat,band,'_meancombined') #+PSF_smear
            print(ref_of_sci[star])
            PSF,n= Loadimagespsf(ref_of_sci[star],'/media/kateryna/Data_Lin/PhD/SPHERE_reduction_data/paper3/filtered/'+ref_of_sci[star]+'/',band)

            print(n)

            a=int(420)#int(n/2.-20*ps)
            b=int(n-420)#int(n/2.+20*ps)
            psf=PSF[a:b,a:b]
            image=image#[a:b,a:b]

            x = np.linspace(-n/2., n/2., num=n)
            y = np.linspace(-n/2., n/2., num=n)
            x0 = 0.5
            y0 = 0.5
            x = x-x0
            y = y-y0
            X, Y = np.meshgrid(x, y)
            R = np.sqrt(X**2 + Y**2)

            lim=100

            plotImage(image, lim)
            plt.savefig(dirName+ star +"_"+band+"_"+fittype+"_image.png",bbox_inches='tight', pad_inches=0.1)
            plt.close()



            plotImage(psf, lim)
            plt.savefig(dirName+star +"_"+band+ "_PSF.png",bbox_inches='tight', pad_inches=0.1)
            plt.close()
            #adc=int(n/2.-10*ps)
            #bdc=int(n/2.+10*ps)
            psf_fliped=np.flip(psf)

            decon = np.copy(image)  # Create starting file for first iteration of deconvolution.  For the very first step, we use the original file as it’s own deconvolution
            for i in range(0, N_decon):  # now iterate
                print ('deconvolution step ' + str(i))  # just some control output

                decon = decon *(convolve_fft(image / convolve_fft(decon, psf), psf_fliped))# (convolve_fft(image / convolve_fft(decon, psf, boundary='wrap'), psf_fliped, boundary='wrap')) 
                plotImage(decon, lim)

                if i<=9:
                    plt.savefig(dirName+star +"_"+band+'_0'+str(i) +"_"+fittype+"_decon2d.png",bbox_inches='tight', pad_inches=0.1)
                else: 
                    plt.savefig(dirName+star +"_"+band+'_'+str(i) +"_"+fittype+"_decon2d.png",bbox_inches='tight', pad_inches=0.1)
                plt.close()
                if i==0:
                    deconvolved1=decon
                    crit=1

                if i>=1:
                    crit=compare(decon[a:b,a:b],deconvolved1[a:b,a:b])        
                    #crit = np.sum(ImageChops.difference(decon[adc:bdc,adc:bdc],deconvolved1[adc:bdc,adc:bdc]))/np.sum(decon[adc:bdc,adc:bdc])
                    #crit=abs(np.sum((decon[adc:bdc,adc:bdc]-deconvolved1[adc:bdc,adc:bdc])**2.)/np.sum(decon[adc:bdc,adc:bdc]**2))
                print(crit)
                deconvolved1=decon


                if fittype!='PSF':
                    if (crit<critlim):
                        N_final=i
                        break        


            # This above is the implementation of the basic equation for the Richardson-Lucy deconvolution.
            # Note the np.fliplr and np.flipud functions which are required due to the quirks of numerical Fourier transformation.
            # I don’t remember the details of the specific implementation here, e.g., why I had to flip for the outer convolution but not for the inner one.
            # Probably becomes clear from a close look at the documentation of the convolve2d function.


            out_fits = fits.HDUList(fits.PrimaryHDU(decon))                  # create output fits structure
            out_fits.writeto(dirName+star+'_'+band+'_decon.fits', overwrite = True)                       # write output

V709_Car
Directory  /media/kateryna/Data_Lin/PhD/SPHERE_reduction_data/paper3/Deconvolution_corr_tel+unres/  already exists
Directory  /media/kateryna/Data_Lin/PhD/SPHERE_reduction_data/paper3/Deconvolution_corr_tel+unres/V709_Car/  already exists
/media/kateryna/Data_Lin/PhD/SPHERE_reduction_data/paper3/Unres+PSFcorr/V709_Car/ V709_Car_I_I_meancombined.fits
V709_Car_I_I_meancombined.fits
1.0402395
HD94680
1024
1024
deconvolution step 0
1
deconvolution step 1
0.24293165289781193
deconvolution step 2
0.1650231410651004
deconvolution step 3
0.12480141107426396
deconvolution step 4
0.10007402038208939
deconvolution step 5
0.08368850609357953
deconvolution step 6
0.07191421265055177
deconvolution step 7
0.06302778568735205
deconvolution step 8
0.056061790139306344
deconvolution step 9
0.050470530158136136
deconvolution step 10
0.0458738786465877
/media/kateryna/Data_Lin/PhD/SPHERE_reduction_data/paper3/Unres+PSFcorr/V709_Car/ V709_Car_I_Q_phi_corr_tel+unres.fits
V709_Car_I_Q_phi_corr_tel+u

/tmp/ipykernel_21485/4227499134.py:87: RuntimeWarning: divide by zero encountered in divide
  decon = decon *(convolve_fft(image / convolve_fft(decon, psf), psf_fliped))# (convolve_fft(image / convolve_fft(decon, psf, boundary='wrap'), psf_fliped, boundary='wrap'))


1
deconvolution step 1
0.19730374088987537
deconvolution step 2
0.12387645711701942
deconvolution step 3
0.08878676159564579
deconvolution step 4
0.0689888632851677
deconvolution step 5
0.05659639884651288
deconvolution step 6
0.04826681159947145
/media/kateryna/Data_Lin/PhD/SPHERE_reduction_data/paper3/Unres+PSFcorr/V709_Car/ V709_Car_I_PI_corr_tel+unres.fits
V709_Car_I_PI_corr_tel+unres.fits
0.046302497
HD94680
1024
1024
deconvolution step 0
1
deconvolution step 1
0.1872891137034409
deconvolution step 2
0.11502350444714252
deconvolution step 3
0.08148507158727536
deconvolution step 4
0.06308858345003411
deconvolution step 5
0.05193936321197628
deconvolution step 6
0.04461698952123334
Directory  /media/kateryna/Data_Lin/PhD/SPHERE_reduction_data/paper3/Deconvolution_corr_tel+unres/  already exists
Directory  /media/kateryna/Data_Lin/PhD/SPHERE_reduction_data/paper3/Deconvolution_corr_tel+unres/V709_Car/  already exists
Directory  /media/kateryna/Data_Lin/PhD/SPHERE_reduction_data/pape